# 双方向LSTMによる品詞のタグ付け

---
## 目的
双方向LSTM (Bi-direcitional LSTM) を用いて英単語に対する品詞をタグ付け (POS tagging) を行う．


## 対応するチャプター
* 10.3: 双方向RNN


## データのダウンロード
実習に必要なデータをダウンロードします．
下記のコードを実行してデータのダウンロードを行ってください．

In [ ]:
!wget http://www.mprg.cs.chubu.ac.jp/~hirakawa/share/tutorial_data/pos-tagging_data.zip
!unzip -q -o pos-tagging_data.zip
!ls ./pos-tagging_data

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
from time import time
import numpy as np
import json

import chainer
from chainer import cuda
from chainer import Variable
import chainer.functions as F
import chainer.links as L
from chainer.optimizer_hooks import GradientClipping

## GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU avilability: True`と表示されれば，GPUを使用した計算をChainerで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

## データセットのダウンロード



In [ ]:
def download_text_dataset(input_filename):
    downloaded = []
    with open(input_filename) as f:
        for s in f.readlines():
            downloaded.append( np.array(list(map(int, s.strip().split(' '))), dtype=np.int32) )
    return downloaded

train_sentence = download_text_dataset("pos-tagging_data/train_sentence.txt")
train_tags = download_text_dataset("pos-tagging_data/train_tags.txt")
test_sentence = download_text_dataset("pos-tagging_data/test_sentence.txt")
test_tags = download_text_dataset("pos-tagging_data/test_tags.txt")
        
with open("pos-tagging_data/vocab.json") as f:
    vocab = json.load(f)
    vocab = {v:k for k, v in vocab.items()}
    
with open("pos-tagging_data/tags.json") as f:
    tags = json.load(f)
    tags = {v:k for k, v in tags.items()}

## ネットワークモデルの定義
Bidirectional LSTMを用いて，品詞タグ付けを行うためのネットワークを定義します．

In [ ]:
def sequence_embed(embed, xs):
    x_len = [len(x) for x in xs]
    x_section = np.cumsum(x_len[:-1])
    ex = embed(F.concat(xs, axis=0))
    exs = F.split_axis(ex, x_section, 0)
    return exs

class BiLSTM(chainer.Chain):
    
    def __init__(self, n_vocab, n_tags, n_layers, n_units):
        super(BiLSTM, self).__init__()
        with self.init_scope():
            self.embed = L.EmbedID(n_vocab, n_units)
            self.bi_lstm = L.NStepBiLSTM(n_layers, n_units, n_units, 0.1)
            self.output = L.Linear(2 * n_units, n_tags)
    
    def forward(self, xs):
        xs = [x[::-1] for x in xs]
        exs = sequence_embed(self.embed, xs)
        
        hx, cx, os = self.bi_lstm(None, None, exs)
        h = self.output(F.concat(os, axis=0))
        return h        

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．
ここでは，GPUで学習を行うために，modelをGPUに送るto_gpu関数を利用しています．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します．また，学習率を0.01として引数に与えます．そして，最適化方法のsetup関数にネットワークモデルを与えます．

In [ ]:
num_vocab = len(vocab)
num_tags = len(tags)

model = BiLSTM(num_vocab, num_tags, 2, 1024)
model.to_gpu()

optimizer = chainer.optimizers.MomentumSGD(lr=0.1, momentum=0.9)
optimizer.setup(model)
optimizer.add_hook(GradientClipping(5.0))

## 学習

学習を実行します．

In [ ]:
xp = cuda.cupy

# ミニバッチサイズ・エポック数．学習データ数の設定
batch_size = 32
epoch_num = 100
train_data_num = len(train_sentence)
num_iter_per_epoch = int(train_data_num / batch_size)

start = time()
for epoch in range(1, epoch_num + 1):
    sum_loss = 0
    
    perm = np.random.permutation(train_data_num)
    
    for i in range(0, train_data_num, batch_size):
        x = [ Variable(cuda.to_gpu(np.array(train_sentence[ii], dtype=np.int32))) for ii in perm[i:i+batch_size] ]
        t = [ Variable(cuda.to_gpu(np.array(train_tags[ii], dtype=np.int32))) for ii in perm[i:i+batch_size] ]

        y = model(x)
        
        loss = F.softmax_cross_entropy(y, F.concat(t, axis=0))
        
        sum_loss += loss.data
        
        optimizer.target.cleargrads()
        loss.backward()
        optimizer.update()
    
    elapsed_time = time() - start
    print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch,
                                                              sum_loss/num_iter_per_epoch,
                                                              elapsed_time))
    
    if epoch % 50 == 0:
        model.to_cpu()
        chainer.serializers.save_npz("bilstm-%03d.npz" % epoch, model)
        model.to_gpu()

## テスト

学習後のネットワークを用いて，翻訳を行います．

In [ ]:
with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):

    for i_test in range(2):
        x = [Variable(cuda.to_gpu(np.array(test_sentence[i_test], dtype=np.int32)))]
        t = np.array(test_tags[i_test], dtype=np.int32)

        y = model(x)
        pred = F.argmax(y, axis=1)
        
        input_sentence = [vocab[i] for i in cuda.to_cpu(x[0].data)]
        pred_tags = [tags[i] for i in cuda.to_cpu(pred.data)]
        true_tags = [tags[i] for i in t]

        print("input sentence:", " ".join(input_sentence))
        print("predicted POS :", " ".join(pred_tags))
        print("true POS      :", " ".join(true_tags) + "\n\n")

## テスト（学習済みモデル）

学習には時間を要するため，下記のコードでは学習済みのモデルを読み込んで，学習後のネットワークでの翻訳結果を確認します．
保存したモデルパラメータを読み込んで，テストデータの翻訳を行います．

In [ ]:
num_vocab = len(vocab)
num_tags = len(tags)
pretrained_model = BiLSTM(num_vocab, num_tags, 2, 1024)

chainer.serializers.load_npz("pos-tagging_data/bilstm.npz", pretrained_model)
pretrained_model.to_gpu()


with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):

    for i_test in range(2):
        x = [Variable(cuda.to_gpu(np.array(test_sentence[i_test], dtype=np.int32)))]
        t = np.array(test_tags[i_test], dtype=np.int32)

        y = pretrained_model(x)
        pred = F.argmax(y, axis=1)
        
        input_sentence = [vocab[i] for i in cuda.to_cpu(x[0].data)]
        pred_tags = [tags[i] for i in cuda.to_cpu(pred.data)]
        true_tags = [tags[i] for i in t]

        print("input sentence:", " ".join(input_sentence))
        print("predicted POS :", " ".join(pred_tags))
        print("true POS      :", " ".join(true_tags) + "\n\n")